In [ ]:
import osmnx as ox

place_name = "Coimbatore, India"
tags = {"amenity": "police"}  # Fetch police station locations

# Correct function
gdf_police = ox.features_from_place(place_name, tags=tags)
display(gdf_police)
gdf_police.to_excel("coimbatore_police.xlsx", index=False)


geometry  \
element id                                                               
node    247177690                            POINT (76.94124 11.08207)   
        308446629                            POINT (77.00835 10.65712)   
        308449858                            POINT (77.01847 10.65657)   
        803271976                            POINT (76.96818 10.99527)   
        803272013                              POINT (76.96778 10.999)   
        1051328308                             POINT (76.96153 10.989)   
        1051328344                           POINT (76.96835 11.01557)   
        1051328360                            POINT (76.96831 11.0153)   
        1051328368                           POINT (76.94904 10.91145)   
        1153096222                           POINT (76.95745 10.32894)   
        1423854834                           POINT (76.95959 11.23952)   
        1470432055                           POINT (76.97132 11.00169)   
        1471854713                            POINT (76.9516 11.02663)   
        1474350853                           POINT (77.00271 11.07473)   
        1474518443                           POINT (76.99787 10.99758)   
        1474803271                           POINT (76.95913 10.99248)   
        1680232798                           POINT (76.93545 10.98838)   
        3571694093                           POINT (76.97157 11.02503)   
        4450449989                           POINT (76.96393 10.98757)   
        5338402894                           POINT (76.75368 10.94195)   
        5970940185                           POINT (76.94119 11.08217)   
        11411647470                           POINT (76.9831 10.54017)   
        12122699748                          POINT (77.10143 11.23155)   
        12473835683                          POINT (77.01016 11.02536)   
        12473896167                           POINT (76.9781 10.96776)   
        12473897891                          POINT (76.99167 11.00283)   
        12473901192                          POINT (76.96826 11.01459)   
        12473949936                           POINT (77.02884 11.0026)   
        12473970775                           POINT (76.96894 11.0253)   
        12473983454                          POINT (76.94124 11.08194)   
way     130035512    POLYGON ((76.94317 10.94, 76.94288 10.93989, 7...   
        133585587    POLYGON ((76.96793 11.00079, 76.96835 11.00089...   
        133739505    POLYGON ((77.00263 11.07466, 77.00265 11.0748,...   
        295103984    POLYGON ((77.12052 11.02494, 77.12043 11.02493...   
        580408779    POLYGON ((76.89485 10.8755, 76.89521 10.87567,...   
        685342422    POLYGON ((76.89494 11.04516, 76.89482 11.04521...   
        923519789    POLYGON ((76.95151 11.0267, 76.95183 11.0267, ...   
        1149353353   POLYGON ((77.02956 11.00041, 77.02956 11.00047...   
        1149353354   POLYGON ((77.02955 11.00017, 77.02968 11.00021...   
        1149353355   POLYGON ((77.0296 11.00006, 77.02973 11.0001, ...   
        1149353356   POLYGON ((77.02963 10.99993, 77.02977 10.99997...   
        1149353357   POLYGON ((77.02978 11.00019, 77.02981 11.00016...   
        1237329086   POLYGON ((77.00624 11.0412, 77.00624 11.04136,...   
        1302295330   POLYGON ((77.18016 11.1094, 77.18015 11.10929,...   
        1348530473   POLYGON ((77.03768 10.91758, 77.03759 10.91747...   

                    amenity                   name:en  \
element id                                              
node    247177690    police                       NaN   
        308446629    police                       NaN   
        308449858    police                       NaN   
        803271976    police                       NaN   
        803272013    police                       NaN   
        1051328308   police                       NaN   
        1051328344   police                       NaN   
        1051328360   police                       NaN   
        10513

In [35]:
import osmnx as ox

place_name = "Coimbatore, India"
tags = {"amenity": "police"}  # Fetch police station locations

# Fetch police station data
gdf_police = ox.features_from_place(place_name, tags=tags)

# Extract OSM node IDs as a list
node_ids = gdf_police.index.tolist()

# Print the list of node IDs
print(node_ids)


[('node', 247177690), ('node', 308446629), ('node', 308449858), ('node', 803271976), ('node', 803272013), ('node', 1051328308), ('node', 1051328344), ('node', 1051328360), ('node', 1051328368), ('node', 1153096222), ('node', 1423854834), ('node', 1470432055), ('node', 1471854713), ('node', 1474350853), ('node', 1474518443), ('node', 1474803271), ('node', 1680232798), ('node', 3571694093), ('node', 4450449989), ('node', 5338402894), ('node', 5970940185), ('node', 11411647470), ('node', 12122699748), ('node', 12473835683), ('node', 12473896167), ('node', 12473897891), ('node', 12473901192), ('node', 12473949936), ('node', 12473970775), ('node', 12473983454), ('way', 130035512), ('way', 133585587), ('way', 133739505), ('way', 295103984), ('way', 580408779), ('way', 685342422), ('way', 923519789), ('way', 1149353353), ('way', 1149353354), ('way', 1149353355), ('way', 1149353356), ('way', 1149353357), ('way', 1237329086), ('way', 1302295330), ('way', 1348530473)]


In [17]:
G = ox.graph_from_place("Coimbatore, India", network_type="drive")
boundary_gdf = ox.geocode_to_gdf("Coimbatore, India")
import leafmap.kepler as leafmap

m = leafmap.Map(center=[11.0168, 76.9558], zoom=12)
m.add_gdf(boundary_gdf, layer_name="Coimbatore Boundary")
m.add_graph(G, layer_name="Road Network")
# or just show the boundary
m.add_gdf(
    boundary_gdf, 
    layer_name="Boundary",
    style={"color": "#FF0000", "fillOpacity": 0.1}
)



AttributeError: 'Map' object has no attribute 'add_graph'